# Markdown Parser

마크다운으로 입력된 내용을 한글 형식에 맞춰서 작성이 되도록 하려고 합니다.

전체 표준을 다 반영하긴 어렵겠지만, 아래와 같이 일부 항목을 반영하고자 합니다.

In [16]:
import re

# 구조 토큰의 종류 정의
STRUCTURE_TOKENS = [
    ('HEADER1', r'(^|\n)# (.*?)(\n|$)'),
    ('HEADER2', r'(^|\n)## (.*?)(\n|$)'),
    ('HEADER3', r'(^|\n)### (.*?)(\n|$)'),
    ('HEADER4', r'(^|\n)#### (.*?)(\n|$)'),
    ('HEADER5', r'(^|\n)##### (.*?)(\n|$)'),
    ('HEADER6', r'(^|\n)###### (.*?)(\n|$)'),
    ('LIST_ITEM', r'(^|\n)( *)(-|\+|\*) (.*?)(\n|$)'),
    ('ORDERED_LIST_ITEM', r'(^|\n)( *)(\d+)\. (.*?)(\n|$)'),
    ('TABLE_SEPARATOR', r'(^|\n)\|[-\s|]*?\|(\n|$)'),
    ('TABLE_ROW', r'(^|\n)\|.*?\|(\n|$)'),
    ('NEWLINE', r'\n'),
    ('TEXT', r'[^\n]+')
]

# 구조 토크나이저 함수 구현
def tokenize_structure(text):
    tokens = []
    while text:
        for token_type, pattern in STRUCTURE_TOKENS:
            match = re.match(pattern, text)
            if match:
                if token_type in ['LIST_ITEM', 'ORDERED_LIST_ITEM']:
                    indent = len(match.group(2))
                    tokens.append((token_type, indent, match.group(4).strip()))
                else:
                    tokens.append((token_type, match.group().strip()))
                text = text[match.end():]
                break
        else:
            # 매칭되는 패턴이 없는 경우 텍스트에서 첫 글자를 잘라냄
            text = text[1:]
    return tokens


In [17]:
# 서식 토큰의 종류 정의
FORMATTING_TOKENS = [
    ('BOLD', r'\*\*(.*?)\*\*'),
    ('ITALIC', r'\*(.*?)\*'),
    ('TEXT', r'[^#\*\n]+')
]

# 서식 토크나이저 함수 구현
def tokenize_formatting(text):
    tokens = []
    while text:
        for token_type, pattern in FORMATTING_TOKENS:
            match = re.match(pattern, text)
            if match:
                tokens.append((token_type, match.group().strip()))
                text = text[match.end():]
                break
        else:
            # 매칭되는 패턴이 없는 경우 텍스트에서 첫 글자를 잘라냄
            text = text[1:]
    return tokens


In [18]:
# 테스트용 마크다운 텍스트
markdown_text = """
# Hello World

This is a **bold** text and this is *italic* text.

- Item 1
  - Subitem 1.1 with **bold** text
  - Subitem 1.2 with *italic* text
    - Subsubitem 1.2.1 with **bold** and *italic* text
- Item 2

1. First item with **bold** text
2. Second item with *italic* text
  1. Subitem 2.1 with **bold** and *italic* text
  2. Subitem 2.2

| Header1 | Header2 |
| ------- | ------- |
| Data1 with **bold** | Data2 with *italic* |
| Data3 | Data4 with **bold** and *italic* |
"""

# 토큰화 및 파싱 실행
structure_tokens = tokenize_structure(markdown_text)

for token in structure_tokens:
  print(token)

('HEADER1', '# Hello World')
('NEWLINE', '')
('TEXT', 'This is a **bold** text and this is *italic* text.')
('NEWLINE', '')
('LIST_ITEM', 0, 'Item 1')
('LIST_ITEM', 2, 'Subitem 1.1 with **bold** text')
('LIST_ITEM', 2, 'Subitem 1.2 with *italic* text')
('LIST_ITEM', 4, 'Subsubitem 1.2.1 with **bold** and *italic* text')
('LIST_ITEM', 0, 'Item 2')
('ORDERED_LIST_ITEM', 0, 'First item with **bold** text')
('ORDERED_LIST_ITEM', 0, 'Second item with *italic* text')
('ORDERED_LIST_ITEM', 2, 'Subitem 2.1 with **bold** and *italic* text')
('ORDERED_LIST_ITEM', 2, 'Subitem 2.2')
('TABLE_ROW', '| Header1 | Header2 |')
('TABLE_SEPARATOR', '| ------- | ------- |')
('TABLE_ROW', '| Data1 with **bold** | Data2 with *italic* |')
('TABLE_ROW', '| Data3 | Data4 with **bold** and *italic* |')
